In [1]:
import modin.pandas as pd

import geopandas as gpd
import shapely

import matplotlib.pyplot as plt
import matplotlib.colors as colors

from modin.config import ProgressBar
from tqdm.auto import tqdm
# ProgressBar.enable()

In [29]:
tokens = pd.read_parquet("tokens.parquet")

In [28]:
print("Tokens:", tokens["wforms"].map(len).sum())
print("Unique tokens:", (~tokens["wforms"].explode().dropna().duplicated()).sum())
print("Tokens (prefiltered):", tokens["wforms_new"].map(len).sum())
print("Unique tokens (prefiltered):", (~tokens["wforms_new"].explode().dropna().duplicated()).sum())

Tokens: 71458267
Unique tokens: 925843
Tokens (prefiltered): 6538326
Unique tokens (prefiltered): 745121
